In [31]:
# FX_DAILY USD/EUR - daily
# Crude oil prices - daily
# REAL_GDP - quarterly
# TREASURY_YIELD (Maturity = 3M, 10Y(10Y-3M), 30Y(30Y-3M)) - daily
# FEDERAL_FUNDS_RATE - daily
# CPI - monthly
# INFLATION - monthly
# UNEMPLOYMENT - monthly
# Technical indicator - own functions derived from closing price of FX_DAILY

In [30]:
import requests
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

# Function to fetch data from Alpha Vantage
def fetch_alpha_vantage_data(function, interval=None, maturity=None, datatype='json', **params):
    base_url = "https://www.alphavantage.co/query"
    api_key = "U025297QMVF56UD3"  # Replace with your actual API key
    params['apikey'] = api_key
    params['function'] = function
    if interval:
        params['interval'] = interval
    if maturity:
        params['maturity'] = maturity
    if datatype:
        params['datatype'] = datatype
    response = requests.get(base_url, params=params)
    if datatype == 'json':
        data = response.json()
    else:
        data = response.text
    return data

# Function to filter data for the last 20 years
def filter_last_20_years(data, date_key):
    twenty_years_ago = datetime.now() - relativedelta(years=20)
    if isinstance(data, list):  # When data is a list of dictionaries (e.g., 'data' key format)
        filtered_data = [entry for entry in data if datetime.strptime(entry[date_key], '%Y-%m-%d') > twenty_years_ago]
    else:  # When data is a dictionary (e.g., 'Time Series (Daily)' format)
        filtered_data = {date: values for date, values in data.items() if datetime.strptime(date, '%Y-%m-%d') > twenty_years_ago}
    return filtered_data

# List of datasets to fetch
datasets = [
    {'name': 'BRENT', 'function': 'BRENT', 'params': {'interval': 'daily'}, 'date_key': 'date'},
    {'name': 'REAL_GDP', 'function': 'REAL_GDP', 'params': {}, 'date_key': 'date'},
    {'name': 'TREASURY_YIELD_3M', 'function': 'TREASURY_YIELD', 'params': {'interval': 'daily', 'maturity': '3month'}, 'date_key': 'date'},
    {'name': 'TREASURY_YIELD_10Y', 'function': 'TREASURY_YIELD', 'params': {'interval': 'daily', 'maturity': '10year'}, 'date_key': 'date'},
    {'name': 'TREASURY_YIELD_30Y', 'function': 'TREASURY_YIELD', 'params': {'interval': 'daily', 'maturity': '30year'}, 'date_key': 'date'},
    {'name': 'FEDERAL_FUNDS_RATE', 'function': 'FEDERAL_FUNDS_RATE', 'params': {'interval': 'daily'}, 'date_key': 'date'},
    {'name': 'CPI', 'function': 'CPI', 'params': {}, 'date_key': 'date'},
    {'name': 'INFLATION', 'function': 'INFLATION', 'params': {}, 'date_key': 'date'},
    {'name': 'UNEMPLOYMENT', 'function': 'UNEMPLOYMENT', 'params': {}, 'date_key': 'date'},
    {'name': 'SP500', 'function': 'TIME_SERIES_DAILY', 'params': {'symbol': 'SPY', 'outputsize': 'full'}, 'date_key': None},
    {'name': 'EURO_STOXX_50', 'function': 'TIME_SERIES_DAILY', 'params': {'symbol': 'STOXX50E', 'outputsize': 'full'}, 'date_key': None}
]

# Loop through each dataset, fetch and save the data
for dataset in datasets:
    name = dataset['name']
    function = dataset['function']
    params = dataset['params']
    date_key = dataset['date_key']
    
    print(f"Fetching data for {name}...")
    data = fetch_alpha_vantage_data(function, **params)
    time.sleep(12)  # To avoid hitting rate limits
    
    # For TIME_SERIES_DAILY, the data key is 'Time Series (Daily)'
    if function == 'TIME_SERIES_DAILY' and 'Time Series (Daily)' in data:
        raw_data = data['Time Series (Daily)']
        filtered_data = filter_last_20_years(raw_data, date_key=None)
        
        # Convert filtered data to a DataFrame
        data_df = pd.DataFrame.from_dict(filtered_data, orient='index')
        
        # Save the DataFrame to a CSV file
        csv_filename = f'{name.lower()}_filtered.csv'
        data_df.to_csv(csv_filename, index=True)
        print(f"{name} data for the last 20 years has been downloaded and saved to '{csv_filename}'.")
    elif 'data' in data:
        raw_data = data['data']
        filtered_data = filter_last_20_years(raw_data, date_key)
        
        # Convert filtered data to a DataFrame
        data_df = pd.DataFrame(filtered_data)
        
        # Save the DataFrame to a CSV file
        csv_filename = f'{name.lower()}_filtered.csv'
        data_df.to_csv(csv_filename, index=False)
        print(f"{name} data for the last 20 years has been downloaded and saved to '{csv_filename}'.")
    else:
        print(f"Key 'data' or 'Time Series (Daily)' not found in response for {name}. Full response: {data}")

print("All data series have been fetched and saved.")


Fetching data for BRENT...
BRENT data for the last 20 years has been downloaded and saved to 'brent_filtered.csv'.
Fetching data for REAL_GDP...
REAL_GDP data for the last 20 years has been downloaded and saved to 'real_gdp_filtered.csv'.
Fetching data for TREASURY_YIELD_3M...
TREASURY_YIELD_3M data for the last 20 years has been downloaded and saved to 'treasury_yield_3m_filtered.csv'.
Fetching data for TREASURY_YIELD_10Y...
TREASURY_YIELD_10Y data for the last 20 years has been downloaded and saved to 'treasury_yield_10y_filtered.csv'.
Fetching data for TREASURY_YIELD_30Y...
TREASURY_YIELD_30Y data for the last 20 years has been downloaded and saved to 'treasury_yield_30y_filtered.csv'.
Fetching data for FEDERAL_FUNDS_RATE...
FEDERAL_FUNDS_RATE data for the last 20 years has been downloaded and saved to 'federal_funds_rate_filtered.csv'.
Fetching data for CPI...
CPI data for the last 20 years has been downloaded and saved to 'cpi_filtered.csv'.
Fetching data for INFLATION...
INFLATIO